In [7]:
import sys
sys.path.append('/home/UR/kontaous/Bureau/Stage/')
from Analyses.traitement import *
import pandas as pd
import numpy as np
from Analyses.traitement import *


In [26]:
tracking = pd.read_csv("../../../Tracking_data/MontPellier(2)_(2)HAC.csv")
events = pd.read_csv("../../Data/match_data/match_Montpellier.csv")
tracking, events = prepare_data(tracking, events)
attributs = ['datetime', 'team.name', 'period', 'player.name', 'position.name', 'location.x', 'location.y', 'pass.recipient.name', 'pass.length', 'pass.outcome.name', 'duel.type.name', 'duel.outcome.name', 'interception.outcome.name', 'foul_committed.type.name', 'ball_receipt.outcome.name', 'shot.end_location.x', 'shot.end_location.y', 'shot.outcome.name', 'dribble.outcome.name',]
df = events[attributs].copy()
df.reset_index(drop=True, inplace=True)
df.index = df.index + 1
position_mapping = {
    'Defender': ['right back', 'left back', 'center back', 'right center back', 'left center back', 'right wing back', 'left wing back'],
    'Midfielder': ['center attacking midfield', 'center defensive midfield', 'left center midfield', 'right center midfield', 'left defensive midfield', 'right defensive midfield'],
    'Forward': ['center forward', 'left center forward', 'right center forward', 'right wing', 'left wing']
}
inverse_mapping = {pos: general_pos for general_pos, positions in position_mapping.items() for pos in positions}
df['position.name'] = df['position.name'].fillna('').str.lower()
df['general_position'] = df['position.name'].map(inverse_mapping).fillna(df['position.name'])
df['position.name'] = df['general_position']
df = df.drop(columns=['general_position'])

df

,datetime,team.name,period,player.name,position.name,location.x,location.y,pass.recipient.name,pass.length,pass.outcome.name,duel.type.name,duel.outcome.name,interception.outcome.name,foul_committed.type.name,ball_receipt.outcome.name,shot.end_location.x,shot.end_location.y,shot.outcome.name,dribble.outcome.name
1,2023-08-13 00:00:00.000,Montpellier,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-13 00:00:00.000,Le Havre,2,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-13 00:00:00.000,Le Havre,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-13 00:00:00.000,Montpellier,2,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-08-13 00:00:00.000,Le Havre,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3322,2023-08-13 00:52:44.851,Montpellier,1,Joris Chotard,Midfielder,106.7,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3323,2023-08-13 00:52:45.724,Montpellier,1,Joris Chotard,Midfielder,109.9,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Incomplete
3324,2023-08-13 00:52:45.724,Le Havre,1,Issa Soumaré,Forward,10.2,71.2,NaN,NaN,NaN,Tackle,Lost Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3325,2023-08-13 00:52:50.073,Montpellier,1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.dtypes

datetime                     datetime64[ns]
team.name                            object
period                                int64
player.name                          object
position.name                        object
location.x                          float64
location.y                          float64
pass.recipient.name                  object
pass.length                         float64
pass.outcome.name                    object
duel.type.name                       object
duel.outcome.name                    object
interception.outcome.name            object
foul_committed.type.name             object
ball_receipt.outcome.name            object
shot.end_location.x                 float64
shot.end_location.y                 float64
shot.outcome.name                    object
dribble.outcome.name                 object
dtype: object

In [29]:

def evaluate_performance(df, team_name, position):
    # Assurez-vous que 'team.name' et 'position.name' sont des chaînes de caractères
    df['team.name'] = df['team.name'].astype(str)
    df['position.name'] = df['position.name'].astype(str)
    
    # Filtrer les données pour l'équipe et la position
    df_filtered = df[(df['team.name'] == team_name) & (df['position.name'] == position)]
    
    if position == 'Defender':
        stats = df_filtered.groupby(['team.name', 'player.name']).agg(
            passes_effectuees=('pass.length', 'count'),
            longueur_moyenne_passes=('pass.length', 'mean'),
            duels=('duel.type.name', 'count'),
            duels_gagnes=('duel.outcome.name', lambda x: (x.isin(['Success Out', 'Success In Play', 'Won']).sum())),
            duels_perdus=('duel.outcome.name', lambda x: (x.isin(['Lost Out', 'Lost In Play']).sum())),
            interceptions_reussies=('interception.outcome.name', lambda x: (x == 'Won').sum())
        ).reset_index()
        
        stats.columns = ['Équipe', 'Joueur', 'Passes effectuées', 'Longueur moyenne des passes', 'Duels', 'Duels gagnés', 'Duels perdus', 'Interceptions réussies']
    
    elif position == 'Midfielder':
        stats = df_filtered.groupby(['team.name', 'player.name']).agg(
            passes_effectuees=('pass.length', 'count'),
            longueur_moyenne_passes=('pass.length', 'mean'),
            passes_progressives=('pass.recipient.name', 'count'),
            dribbles_reussis=('dribble.outcome.name', lambda x: (x == 'Complete').sum()),
            ballons_interceptes_succes=('ball_receipt.outcome.name', lambda x: (x == 'Incomplete').sum())
        ).reset_index()
        
        stats.columns = ['Équipe', 'Joueur', 'Passes effectuées', 'Longueur moyenne des passes', 'Passes progressives', 'Dribbles réussis', 'Ballons interceptés avec succès']
    
    elif position == 'Forward':
        stats = df_filtered.groupby(['team.name', 'player.name']).agg(
            passes_effectuees=('pass.length', 'count'),
            longueur_moyenne_passes=('pass.length', 'mean'),
            tirs=('shot.end_location.x', 'count'),
            buts=('shot.outcome.name', lambda x: (x == 'Goal').sum()),
            tirs_cadres=('shot.outcome.name', lambda x: (x == 'Saved').sum()),
            tirs_non_cadres=('shot.outcome.name', lambda x: (x == 'Off T').sum()),
            dribbles_reussis=('dribble.outcome.name', lambda x: (x == 'Complete').sum())
        ).reset_index()
        
        stats.columns = ['Équipe', 'Joueur', 'Passes effectuées', 'Longueur moyenne des passes', 'Tirs', 'Buts', 'Tirs cadrés', 'Tirs non cadrés', 'Dribbles réussis']
    
    else:
        stats = pd.DataFrame()
    
    return stats

df_defenseurs = evaluate_performance(df, "Montpellier", "Defender")
df_attaquants = evaluate_performance(df, "Montpellier", "Forward")
df_attaquants

,Équipe,Joueur,Passes effectuées,Longueur moyenne des passes,Tirs,Buts,Tirs cadrés,Tirs non cadrés,Dribbles réussis
0,Montpellier,Akor Jerome Adams,10,14.995580,2,2,0,0,0
1,Montpellier,Arnaud Nordin,4,20.994834,0,0,0,0,0
2,Montpellier,Moussa Al Tamari,20,14.064753,3,0,2,1,4
3,Montpellier,Sepe Elye Wahi,6,14.297619,0,0,0,0,1
4,Montpellier,Wahbi Khazri,15,23.137950,5,0,1,3,0


In [11]:
df['interception.outcome.name'].unique()

array([nan, 'Lost Out', 'Lost In Play', 'Won', 'Success In Play'],
      dtype=object)

In [30]:
import dash
from dash import dcc, html, dash_table
import pandas as pd
import plotly.graph_objs as go

def display_player_performance(df, position, players):
    # Filtrer les données en fonction de la position (si nécessaire)
    df_position = df[df['Équipe'] == position] if position else df
    
    # Sélectionner les joueurs spécifiés
    df_selected = df_position[df_position['Joueur'].isin(players)]
    
    # Transformation des données pour le radar chart
    data_radar = {'Metrics': df_selected.columns[2:].tolist()}
    
    for player in players:
        data_radar[player] = df_selected[df_selected['Joueur'] == player].iloc[0, 2:].tolist()
    
    df_radar = pd.DataFrame(data_radar)
    
    # Fonction pour créer le radar chart
    def create_radar_chart(df):
        fig = go.Figure()

        for column in df.columns[1:]:
            fig.add_trace(go.Scatterpolar(
                r=df[column],
                theta=df['Metrics'],
                fill='toself',
                name=column
            ))

        fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, max(df.iloc[:, 1:].max().max(), 10)]
                )
            ),
            showlegend=True,
            title="Comparaison des Performances des Joueurs"
        )

        return fig
    
    app = dash.Dash(__name__)

    app.layout = html.Div([
        html.H1("Visualisation des Performances des Joueurs"),
        dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df_selected.columns],
            data=df_selected.to_dict('records'),
            style_cell={'textAlign': 'center'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ],
            style_as_list_view=True
        ),
        dcc.Graph(
            id='radar-chart',
            figure=create_radar_chart(df_radar)
        )
    ])

    if __name__ == '__main__':
        app.run_server(debug=True, port=8051)



# Exemple d'utilisation de la fonction
display_player_performance(df_attaquants, 'Montpellier', ['Akor Jerome Adams', 'Moussa Al Tamari', 'Arnaud Nordin'])
